In [5]:
from sentence_transformers import SentenceTransformer

import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
import plotly.express as px
import numpy as np
import nltk
import jieba
from arabic_reshaper import reshape
import re



based on : https://pypi.org/project/sentence-transformers/

In [6]:
model = SentenceTransformer("all-MiniLM-L6-v2")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [8]:
df = pd.read_csv('data/test.csv') 

sentences = pd.DataFrame(df)

sentences = sentences.drop_duplicates()

In [ ]:
def preprocess_text(text):
    text = text.lower() 
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text) 
    text = re.sub(r'[^a-zA-Z\s]', '', text)  
    return text


sentences['cleaned_premise'] = sentences['premise'].apply(preprocess_text)
sentences['cleaned_hypothesis'] = sentences['hypothesis'].apply(preprocess_text)
print(sentences['cleaned_premise'])

In [ ]:
embeddings_premises = model.encode(sentences['cleaned_premise'].tolist(), show_progress_bar=True)
embeddings_hypotheses = model.encode(sentences['cleaned_hypothesis'].tolist(), show_progress_bar=True)
print(embeddings.shape)

KeyError: 6